In [6]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)

n = 0
run_sum = 0

for sqrt_value in generator:
    n += 1
    run_sum += sqrt_value
    print(n, ' ', sqrt_value, ' ', run_sum)

1   1.0   1.0
2   1.4142135623730951   2.414213562373095
3   1.7320508075688772   4.146264369941973
4   2.0   6.146264369941973
5   2.23606797749979   8.382332347441762
6   2.449489742783178   10.83182209022494
7   2.6457513110645907   13.47757340128953
8   2.8284271247461903   16.30600052603572
9   3.0   19.30600052603572
10   3.1622776601683795   22.4682781862041
11   3.3166247903554   25.7849029765595
12   3.4641016151377544   29.249004591697254
13   3.605551275463989   32.854555867161245


In [28]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)

{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}


In [29]:
def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)

{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [11]:
import dlt

# define the connection to load to.
# We now use duckdb, but you can switch to Bigquery later
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')


# we can load the next generator to the same or to a different table.
info = generators_pipeline.run(people_1(),
							   table_name="stream_download",
							   write_disposition="replace")

print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.20 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////Users/pavelmatveev/Learning/data-engineering-zoomcamp-2024/dlt_workshop/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707766425.010506 is LOADED and contains no failed jobs


In [18]:
import duckdb
import pandas as pd

/var/folders/rp/d0qnmq2s6d504z6v9nlfbrsh0000gn/T/ipykernel_1343/1292183862.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [24]:
# show the outcome
conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

people_1 = conn.sql("SELECT * FROM stream_download").df()
display(people_1)

people_1 = conn.sql("SELECT sum(age) FROM stream_download").df()
display(people_1)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ stream_download     │
└─────────────────────┘

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707766425.010506,PnG1LnEVQo0Gww,None
1,2,Person_2,27,City_A,1707766425.010506,0CCk3/e/OBfm3w,None
2,3,Person_3,28,City_A,1707766425.010506,VkC47eIB+8Tk6w,None
3,4,Person_4,29,City_A,1707766425.010506,OI97A/wHVHy0yw,None
4,5,Person_5,30,City_A,1707766425.010506,Q6ophg2GpKgT0w,None
5,3,Person_3,33,City_B,1707767072.864764,Qok2wu6MLPAVZg,Job_3
6,4,Person_4,34,City_B,1707767072.864764,Znj2wzwDa8zlZw,Job_4
7,5,Person_5,35,City_B,1707767072.864764,nLqLETNa9+YIUw,Job_5
8,6,Person_6,36,City_B,1707767072.864764,cf3hODqCBhqZRA,Job_6
9,7,Person_7,37,City_B,1707767072.864764,flu4ctyjOJJ5GQ,Job_7


,sum(age)
0,353.0


In [22]:
info = generators_pipeline.run(people_2(),
							   table_name="stream_download",
							   write_disposition="append")

print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.16 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////Users/pavelmatveev/Learning/data-engineering-zoomcamp-2024/dlt_workshop/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707767072.864764 is LOADED and contains no failed jobs


In [32]:
# generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')

# info_1 = generators_pipeline.run(people_1(),
# 							     table_name="stream_download_merge",
# 							     write_disposition="replace")

# print(info_1)


info_2 = generators_pipeline.run(people_2(),
							     table_name="stream_download_merge",
							     write_disposition="merge",
                                 primary_key="id")

print(info_2)

Pipeline dlt_ipykernel_launcher load step completed in 0.29 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////Users/pavelmatveev/Learning/data-engineering-zoomcamp-2024/dlt_workshop/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707767733.225007 is LOADED and contains no failed jobs


In [33]:
# show the outcome
conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

people_1 = conn.sql("SELECT * FROM stream_download_merge").df()
display(people_1)

people_1 = conn.sql("SELECT sum(age) FROM stream_download_merge").df()
display(people_1)

Loaded tables: 


┌───────────────────────┐
│         name          │
│        varchar        │
├───────────────────────┤
│ _dlt_loads            │
│ _dlt_pipeline_state   │
│ _dlt_version          │
│ stream_download       │
│ stream_download_merge │
└───────────────────────┘

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707767583.8643599,WcVR1cwJcBp3jg,None
1,2,Person_2,27,City_A,1707767583.8643599,EzZe0Eg+GkZtgA,None
2,8,Person_8,38,City_B,1707767733.225007,GK5l97wl2YjCNA,Job_8
3,4,Person_4,34,City_B,1707767733.225007,3Ca68RuNAh1RkQ,Job_4
4,5,Person_5,35,City_B,1707767733.225007,3bMGikO1gz/K1A,Job_5
5,7,Person_7,37,City_B,1707767733.225007,inTbiWFY4vPmkQ,Job_7
6,3,Person_3,33,City_B,1707767733.225007,+68UerjyfpKAvQ,Job_3
7,6,Person_6,36,City_B,1707767733.225007,HdrDR5jYc/pleA,Job_6


,sum(age)
0,266.0
